# PyTorch Lesson 3


Things we will learn in this lesson:

    - Multiple Linear Regression
    - how to run and predict the output
    - Logistic regression basics
    - Run logistic regression in pytorch for 0D,1D &2D
    - Binary cross entropy function

#### Multiple linear regression

In this regresion type the input variable has multiple feature columns. 
For eg: in terms of employee salary prediction
        input  -> age, name, years of experience, dept of work, designation
        output -> salary for the employee
        
We will therefore see how the hypothesis function is deviced for such a multiple input linear regression probelem


y = Xw + b

The input vector should be x = 1xD

weight/parameter = Dx1

bias = 1
    
Now lets create an weight vector with bias using "LINEAR module" and then use the same to do the calculation for **Forward pass**

In [6]:
from torch.nn import Linear
import torch

In [3]:
model = Linear(in_features=2, out_features=1)

In [4]:
model.state_dict()

OrderedDict([('weight', tensor([[-0.4651,  0.1902]])),
             ('bias', tensor([-0.4801]))])

In [9]:
x = torch.tensor([[1.,2.],[3.,4.],[1.,-1.]])

In [10]:
yhat = model(x)

In [11]:
yhat

tensor([[-0.5647],
        [-1.1145],
        [-1.1354]], grad_fn=<AddmmBackward>)

Thus Linear function is used to create the yhat output without explicitly coding the equation

> yhat = b + wX

## Custom function for Multiple Linear Regression

In this method we will build a multiple linear regression model by,

- Creating a customer dataset using dataset class
- Creating loss and optimizer object using exitsing packages
- Training regression model

In [16]:
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset

The below class is a custom class inherited from "Dataset" class of torch.

This class is basically to hold the dataset for training and methods to index call and length identification

In [56]:
class dataset(Dataset):
    def __init__(self):
        self.x = torch.zeros(20,2)
        self.x[:,0] = torch.arange(-1,1,0.1)
        self.x[:,1] = torch.arange(-1,1,0.1)
        self.w = torch.tensor([[-10.],[-5.0]])
        self.b = torch.tensor([-1.0])
        
        self.f = torch.mm(self.x,self.w) + self.b
        self.y = self.f + (0.1*torch.randn((self.x.shape[0],1)))
        self.len = self.x.shape[0]
        
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.len

The below class is an inherited class from nn.Module

This class helps us in defining the weights and bias using linear class of torch and leveraging the same for forward pass calculations

In [57]:
class Lin_Reg(nn.Module):
    def __init__(self,in_,out_):
        super(Lin_Reg,self).__init__()
        self.Linear = nn.Linear(in_,out_)
        
    def forward(self,x):
        return self.Linear(x)

In [58]:
data = dataset()
trainloader = DataLoader(dataset=data,batch_size=2)

In [62]:
model = Lin_Reg(2,1)
loss = nn.MSELoss()
print(model.state_dict())
optimizer = optim.SGD(model.parameters(), lr=0.01)

OrderedDict([('Linear.weight', tensor([[ 0.1538, -0.4462]])), ('Linear.bias', tensor([0.3360]))])


Now lets start training the model using the dataset defined for a batch size of 2 and epochs of 100 count

In [64]:
for epoch in range(100):
    for x,y in trainloader:
        yhat = model(x)
        loss_ = loss(yhat,y)
        optimizer.zero_grad()
        loss_.backward()       
        optimizer.step()

#### For multiple output regression

In [92]:
class dataset(Dataset):
    def __init__(self):
        self.x = torch.zeros(20,2)
        self.x[:,0] = torch.arange(-1,1,0.1)
        self.x[:,1] = torch.arange(-1,1,0.1)
        self.w = torch.tensor([[-10.],[-5.0]])
        self.b = torch.tensor([-1.0])
        
        self.f = torch.mm(self.x,self.w) + self.b
        self.y = torch.zeros(20,2)
        self.y[:,0] = (self.f + (0.1*torch.randn((self.x.shape[0],1)))).view(20)
        self.y[:,1] = (self.f + (0.1*torch.randn((self.x.shape[0],1)))).view(20)
        self.len = self.x.shape[0]
        
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.len

In [93]:
data = dataset()
trainloader = DataLoader(dataset=data,batch_size=2)

#Specifing out variable to be 2 so that it takes corresponding y to fit the model and output 2 variables
model = Lin_Reg(2,2)
loss = nn.MSELoss()
print(model.state_dict())
optimizer = optim.SGD(model.parameters(), lr=0.01)

OrderedDict([('Linear.weight', tensor([[0.5148, 0.2370],
        [0.2428, 0.2640]])), ('Linear.bias', tensor([ 0.2955, -0.0827]))])


In [94]:
for epoch in range(100):
    for x,y in trainloader:
        yhat = model(x)
        loss_ = loss(yhat,y)
        optimizer.zero_grad()
        loss_.backward()       
        optimizer.step()

Thus this method will calculate forward pass and generate output for each of the target y respectively

## Logistic Regression

The linear regression steps all applies here. Except that the plane that is formulated distingushes the classes in the target variable

In terms of binary classification model, sigmoid function is passed on to the output z (wx+b) values to find the probability of class 0 and 1

**Sigmoid -->  pushes the value of z between 0 and 1**

to use sigmoid function use **torch.sigmoid()**

In [96]:
class Log_Reg(nn.Module):
    def __init__(self,in_):
        super(Log_Reg,self).__init__()
        self.Linear = nn.Linear(in_,1)#in log reg the output param will always be one
        
    def forward(self,x):
        return torch.sigmoid(self.Linear(x))

**Lets take an input tensor with single scalar value (0 dimension)**

In [98]:
model = Log_Reg(1)
x = torch.tensor([1.0])

yhat  = model.forward(x)
print('The output of single scalar input is:',yhat)

The output of single scalar input is: tensor([0.5048], grad_fn=<SigmoidBackward>)


We can clearly see that the output is between 0 and 1

**Now lets take a multiple sample input of 1D (one feature) **

In [99]:
model = Log_Reg(1)
x =  torch.tensor([[1.],[-5]])

yhat  = model.forward(x)
print('The output of single 1D input is:',yhat)

The output of single 1D input is: tensor([[0.5101],
        [0.9743]], grad_fn=<SigmoidBackward>)


**Finally now we would take a multi sample 2D dimensional exmaple**

In [100]:
model = Log_Reg(2)
x =  torch.tensor([[1.,-1.],[-5,2.]])

yhat  = model.forward(x)
print('The output of single 2D input is:',yhat)

The output of single 2D input is: tensor([[0.6109],
        [0.3818]], grad_fn=<SigmoidBackward>)


Incase of logistic regression we use likelihood from bernouliie distribution

where likelihod the product of probabilities of an outcome given the parameter values to **stay close to ground truth**

so increasing the likelihood is the key to thsi process. Since log is a monotonically increaing function we use it to increase the likelihood

With log the probabilities in product becomes additive in nature, thus -log will cause a concave function thus helping in reducing the loss while choose our parameters for best fit

With the best parameter the lower point of the fucntion has **min loss and max likelihood**

**Binary cross-entropy loss is give by:**

j(w,x) = -1/n sumation(y log yhat + (1-y) log (1-yhat))

In [101]:
loss = nn.BCELoss()